In [17]:
#importing Libraries

import numpy as np
# Powerful n-dimensional arrays. Numerical computing tools
import pandas as pd
# is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool
import seaborn as sns
# Seaborn is a Python data visualization library based on matplotlib.
import matplotlib.pyplot as plt
# Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python
from subprocess import check_output
# The subprocess is used to run new applications or programs through Python code by creating new processes.
# It also helps to obtain the input/output/error pipes as well as the exit codes of various commands
%matplotlib inline
#With this backend, the output of plotting commands is displayed inline within frontends 
#like the Jupyter notebook
#import plotly.offline as py
#plotly enables Python users to create beautiful interactive web-based visualizations
#py.init_notebook_mode(connected=True)
#to plot plotply ..
#import plotly.graph_objs as go
#import plotly.tools as tls
import os
#It imports the OS library, which is mainly useful to do OS stuff (bet that was really helpful to you!)
#OS stuff includes: showing the absolute path of a file
#import gc
#Python's garbage collector runs during program execution and is triggered when an object's reference count reaches zero.
import re
#a regular expression is denoted as RE 
from nltk.corpus import stopwords
# stopwords 
#NLTK is a leading platform for building Python programs to work with human language data. 
import distance
from nltk.stem import PorterStemmer
#The Porter stemming algorithm (or 'Porter stemmer') is a process for removing the commoner morphological 
#and inflexional endings from words in English
from bs4 import BeautifulSoup
#Beautiful Soup is a Python library for pulling data out of HTML and XML files.
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('stopwords')
from os import path
#from PIL import Imagedf = pd.read_csv("minor/DataSet/Quora.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [10]:
df = pd.read_csv("Quora.csv")

In [12]:
df = df.fillna(' ')


In [13]:
if os.path.isfile("df_features_train.csv"):
    df = pd.read_csv("df_features_train.csv",encoding='latin-1')
else:    
    df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count')
    df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')
    df['q1_len'] = df['question1'].str.len()
    df['q2_len'] = df['question2'].str.len()
    
    df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
    df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))
    
    def word_common(row):
        w1 = set(map(lambda word : word.lower().strip(),row['question1'].split(' ')))
        w2 = set(map(lambda word : word.lower().strip(),row['question2'].split(' ')))
        return 1.0 * len(w1&w2)
    df['word_common'] = df.apply(word_common,axis = 1)
    
    def word_total(row):
        w1 = set(map(lambda word : word.lower().strip(),row['question1'].split(' ')))
        w2 = set(map(lambda word : word.lower().strip(),row['question2'].split(' ')))
        return 1.0 * (len(w1) + len(w2))
    df['word_total'] = df.apply(word_total,axis=1)
    
    def word_share(row):
        w1 = set(map(lambda word : word.lower().strip(),row['question1'].split(' ')))
        w2 = set(map(lambda word : word.lower().strip(),row['question2'].split(' ')))
        return 1.0*( len(w1 & w2)/(len(w1)+len(w2)))
    df['word_share'] = df.apply(word_share,axis=1)
    
    df['freq_q1+freq_q2'] = df['freq_qid1'] + df['freq_qid2']
    df['freq_q1-freq_q2'] = abs(df['freq_qid1'] - df['freq_qid2'])
    
    df.to_csv('df_features_train.csv',index='False')
    
df.head()   

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1_len,q2_len,q1_n_words,q2_n_words,word_common,word_total,word_share,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [14]:
def word_first(row):
        w1 = row['question1'].split(' ')[0].lower()
        w2 = row['question2'].split(' ')[0].lower()
        return w1,w2,w1==w2
tup = (df.apply(word_first,axis=1))

In [18]:
STOP_WORDS = stopwords.words("english") # is // the // very  // not  // to  
def preprocess(x):
    x = str(x).lower()
    #stack exch
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    
    # replacing millions 1m and thosands by 1k
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    #######
    pattern = re.compile('\W')

    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
# beauti : beautiful   
    if type(x) == type(''):
        x = porter.stem(x)
        soup = BeautifulSoup(x)
        x = soup.get_text()
        
    return x


In [19]:
def get_features(q1,q2):
    # 10 list that will contain feature 
    token_features = [0.0]*10
    SAFE_DIV = 0.0001
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
# FUZZY - SIMILAR 
    # Q1 NEW YORK MEaTs    fuzzy ratio good
    # Q2 NEW YORK MEATS
    #dISTANCE FEATURE /// EDITS Fuzzy ratio  
    # NO OF EDITS GOOD FUZZY RATIO
    #  0-100
    # Similar to string matching problem learned in dynamic programming
    # string 1 YANKEES , NEW YORK YANKEES 
    # string 2 NEWYORK METS NEW YORK YANKEES
    # fuuzz partial ratio # substring matches
    # token sort ..
    # "csk vs rcb"   
    # "rcb vs csk"   partial ratio ... 
    # csk , vs , rcb # sort     
    # csk vs rcb  # 80 - 90  3 
    # csk vs rcb , had a great match today and ............  200
    # token set ratio
    #t1 = [sorted intersection]
    # t2 = ..... + remaining s1  ->''
    # t3 = ...... + remaining of s2 -> 'had a great match.......'
    # t1 , t2 , t3 ratio similarity ..... 
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features


def get_longest_substr_ratio(a,b):
    strs = list(distance.lcsubstrings(a,b))
    if(len(strs)==0):
        return 0
    else:
        return len(strs[0]) / (min(len(a),len(b))+1)
    
def extract_features(df):
    # preprocessing
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)
    
    print("token features")
    
    token_feature = df.apply((lambda x: get_features(x["question1"],x["question2"])),axis=1)
    
    df['cwc_min'] = list(map(lambda x : x[0], token_feature))
    df['cwc_max'] = list(map(lambda x : x[1], token_feature))
    df['csc_min'] = list(map(lambda x : x[2], token_feature))
    df['csc_max'] = list(map(lambda x : x[3], token_feature))
    df['ctc_min'] = list(map(lambda x : x[4], token_feature))
    df['ctc_max'] = list(map(lambda x : x[5], token_feature))
    df['last_word_eq'] = list(map(lambda x : x[6], token_feature))
    df['first_word_eq'] = list(map(lambda x : x[7], token_feature))
    df['abs_len_diff'] = list(map(lambda x: x[8] , token_feature))
    df['mean_len'] =  list(map(lambda x : x[7], token_feature))
    
    print("fuzzy features..")

    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

In [20]:
if os.path.isfile('minor/Files/nlp_features_train.csv'):
    df = pd.read_csv("minor/Files/nlp_features_train.csv",encoding='latin-1')
    df.fillna('')
else:
    print("Extracting features for train:")
    df = pd.read_csv("Quora.csv")
    df = extract_features(df)
    df.to_csv("Files/nlp_features_train.csv", index=False)
df.head(2)

Extracting features for train:
token features
fuzzy features..


,id,qid1,qid2,question1,question2,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.999980,0.833319,0.999983,0.999983,...,0.785709,0.0,1.0,2.0,1.0,100,93,93,100,0.982759
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.799984,0.399996,0.749981,0.599988,...,0.466664,0.0,1.0,5.0,1.0,86,63,66,75,0.596154
